In [2]:
import pandas as pd
import networkx as nx

df = pd.read_csv("../Generador/Dataframe.csv", index_col = False)
df = df.drop(columns=["Unnamed: 0"])
df

,direccion,tipoVivienda,material_paredes,material_techo,material_piso,cuartos,banos,cocinas,comedores,dormitorios,personas_vivienda,jefe_hogar,departamento,provincia,distrito,date_time,ubigeo,fecha_censo,quien_respondio
0,Calle de Carolina Valle 502,Casa,Ladrillo,Eternit,Ceramica,6,2,1,1,1,2,Graciano Ayuso Mora,PIURA,MORROPON,BUENOS AIRES,2019-02-21 10:25:06,200402,07/07/2017,Graciano Ayuso Mora
1,Pasaje de Luis Miguel Olivera 594,Departamento,Ladrillo,Eternit,Parquet,2,3,1,0,1,2,Flavia Ribes Blanch,AYACUCHO,CANGALLO,PARAS,2010-05-25 02:47:37,50205,05/07/2017,Flavia Ribes Blanch
2,Camino de Delia Royo 485,Apartamento,Concreto,Techo de Paja,Ceramica,5,2,2,1,4,10,Ariel Miranda Cortés,MADRE DE DIOS,MANU,MADRE DE DIOS,2015-11-10 18:06:44,170203,10/07/2017,Ariel Miranda Cortés
3,Plaza Nuria Pera 812,Apartamento,Ladrillo,Techo de Paja,Ceramica,4,2,2,1,1,10,Amando Morante Navarro,SAN MARTIN,HUALLAGA,EL ESLABON,2015-04-23 05:16:18,220403,21/07/2017,Amando Morante Navarro
4,Cañada Olimpia Echevarría 904,Apartamento,Ladrillo,Techo de Paja,Parquet,1,4,1,0,1,2,Reyna Aznar-Alemán,ANCASH,HUAYLAS,PUEBLO LIBRE,1973-09-22 11:11:30,21207,01/07/2017,Reyna Aznar-Alemán
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,Plaza de Candelas Ugarte 686,Apartamento,Concreto,Eternit,Parquet,4,1,2,0,3,10,Noa Villena-Gibert,LIMA,LIMA,EL AGUSTINO,2000-08-11 13:46:31,150111,08/07/2017,Noa Villena-Gibert
49996,Via José Antonio Cañellas 371,Casa,Adobe,Teja,Parquet,6,4,1,2,2,2,Roldán Oliver-Ferrández,LAMBAYEQUE,CHICLAYO,SAÑA,2014-06-11 02:16:42,140115,02/07/2017,Roldán Oliver-Ferrández
49997,Ronda Raquel Patiño 806,Apartamento,Adobe,Techo de Paja,Parquet,1,1,2,2,1,2,José Ángel Conde-Moliner,CAJAMARCA,CHOTA,LAJAS,2015-03-27 04:18:03,60410,18/07/2017,José Ángel Conde-Moliner
49998,C. Juan Antonio Infante 245,Casa,Ladrillo,Teja,Parquet,5,4,1,1,4,1,Pepe Vilar Corbacho,LORETO,MAYNAS,ALTO NANAY,1985-09-06 07:41:51,160102,31/07/2017,Pepe Vilar Corbacho


In [3]:
G = nx.Graph()

G.add_node("Pais.Peru")

def accessDepartamento(dataFramePais, Grafo):
    Departamentos = list(dataFramePais["departamento"].unique())
    for departamento in Departamentos:
        Grafo.add_node("Departamento." + str(departamento), attribute = 'Departamento')
        Grafo.add_edge("Pais.Peru", "Departamento." + str(departamento))
        accessProvincia(dataFramePais, Grafo, departamento)

def accessProvincia(dataFrameDepartamento, Grafo, departamento):
    dataFrameProvincia = df.iloc[list(dataFrameDepartamento["departamento"] == departamento)]
    Provincias = list(dataFrameProvincia["provincia"].unique())
    for provincia in Provincias: 
        Grafo.add_node("Provincia." + str(provincia), attribute = 'Provincia')
        Grafo.add_edge("Departamento." + str(departamento), "Provincia." + str(provincia))
        accessDistrito(dataFrameProvincia, Grafo, provincia)

def accessDistrito(dataFrameProvincia, Grafo, provincia):
    dataFrameDistrito = dataFrameProvincia.iloc[list(dataFrameProvincia["provincia"] == provincia)]
    Distritos = list(dataFrameDistrito["distrito"].unique())
    for distrito in Distritos: 
        Grafo.add_node("Distrito." + str(distrito), attribute = 'Distrito')
        Grafo.add_edge("Provincia." + str(provincia), "Distrito." + str(distrito))
        #accessDireccion(dataFrameDistrito, Grafo, distrito)

def accessDireccion(dataframeDistrito, Grafo, distrito):
    for index, row in dataframeDistrito.iterrows():
                direccion = row["direccion"]
                Grafo.add_node("Direccion." + str(direccion))
                Grafo.nodes[direccion].update(row.to_dict())
                Grafo.add_edge("Distrito." + str(distrito), "Direccion." + str(direccion))
                

                
accessDepartamento(df, G)
nx.write_graphml_lxml(G,"test.graphml")

In [4]:
G.nodes()

NodeView(('Pais.Peru', 'Departamento.PIURA', 'Provincia.MORROPON', 'Distrito.BUENOS AIRES', 'Distrito.SALITRAL', 'Distrito.SAN JUAN DE BIGOTE', 'Distrito.SANTA CATALINA DE MOSSA', 'Distrito.LA MATANZA', 'Distrito.CHALACO', 'Distrito.CHULUCANAS', 'Distrito.MORROPON', 'Distrito.SANTO DOMINGO', 'Distrito.YAMANGO', 'Provincia.HUANCABAMBA', 'Distrito.LALAQUIZ', 'Distrito.HUANCABAMBA', 'Distrito.EL CARMEN DE LA FRONTERA', 'Distrito.SAN MIGUEL DE EL FAIQUE', 'Distrito.HUARMACA', 'Distrito.SONDOR', 'Distrito.SONDORILLO', 'Distrito.CANCHAQUE', 'Provincia.PIURA', 'Distrito.CASTILLA', 'Distrito.LAS LOMAS', 'Distrito.PIURA', 'Distrito.LA UNION', 'Distrito.TAMBO GRANDE', 'Distrito.LA ARENA', 'Distrito.CURA MORI', 'Distrito.CATACAOS', 'Distrito.VEINTISEIS DE OCTUBRE', 'Distrito.EL TALLAN', 'Provincia.AYABACA', 'Distrito.SUYO', 'Distrito.LAGUNAS', 'Distrito.FRIAS', 'Distrito.PACAIPAMPA', 'Distrito.AYABACA', 'Distrito.JILILI', 'Distrito.MONTERO', 'Distrito.PAIMAS', 'Distrito.SAPILLICA', 'Distrito.SICC

In [5]:
G["Pais.Peru"]

#for x in G.edges("Peru"):
#    print(x)

def accessEdges(Grafo, node):
    for nodes in G.edges("Pais.Peru"):
        print(nodes[1])
        for nodenodes in G.edges(nodes[1]):
            print(nodenodes[1])
        print()
        
accessEdges(G, "Peru")

Departamento.PIURA
Pais.Peru
Provincia.MORROPON
Provincia.HUANCABAMBA
Provincia.PIURA
Provincia.AYABACA
Provincia.SULLANA
Provincia.SECHURA
Provincia.PAITA
Provincia.TALARA

Departamento.AYACUCHO
Pais.Peru
Provincia.CANGALLO
Provincia.HUANTA
Provincia.LA MAR
Provincia.VICTOR FAJARDO
Provincia.SUCRE
Provincia.LUCANAS
Provincia.PARINACOCHAS
Provincia.VILCAS HUAMAN
Provincia.HUAMANGA
Provincia.PAUCAR DEL SARA SARA
Provincia.HUANCA SANCOS

Departamento.MADRE DE DIOS
Pais.Peru
Provincia.MANU
Provincia.TAHUAMANU
Provincia.TAMBOPATA

Departamento.SAN MARTIN
Pais.Peru
Provincia.HUALLAGA
Provincia.RIOJA
Provincia.PICOTA
Provincia.LAMAS
Provincia.TOCACHE
Provincia.SAN MARTIN
Provincia.MOYOBAMBA
Provincia.BELLAVISTA
Provincia.MARISCAL CACERES
Provincia.EL DORADO

Departamento.ANCASH
Pais.Peru
Provincia.HUAYLAS
Provincia.HUARI
Provincia.OCROS
Provincia.PALLASCA
Provincia.HUARAZ
Provincia.ANTONIO RAYMONDI
Provincia.SANTA
Provincia.CORONGO
Provincia.BOLOGNESI
Provincia.HUARMEY
Provincia.CARHUAZ
Prov